In [6]:
#Setting all the input variables

storm_name = "dorian"
storm_year = 2019
storm_lowerLat= 40

# How many days before / after the storm dates to get data for graphing
dateExtension = 2

focus_variable = "wind_spd_avg"
var_units = "(m s-1)"

# Time in hours after the first measurement of the storm to observe
storm_time_offset = 8

# Time in hours to go back for a station's measurement
# Change to just have the minimum time be the start of the storm?
storm_time_bounds = 1

# Color map style to use for the output station data
color_map = 'Blues'

# Buoys to exclude (for demoing purposes these either give odd values or overlap with other markers)
#exclude=  ["sma_negl_cartwright_junction_nlqu0004", "SMA_halifax_fairview", "SMA_port_aux_basqes_wharf"]
exclude= []

#interest_variables = ["wind_spd", "wave_ht", "pressure"]
# Categories: Wind, Surface Waves, Pressure

In [29]:
# Getting storm data and matching ERDDAP datasets around the storm (time and space)

import tropycal.tracks as tracks
import pandas as pd
from erddapy import ERDDAP
from datetime import timedelta
import pytz

basin = tracks.TrackDataset(basin='north_atlantic', source='ibtracs')
storm = basin.get_storm((storm_name,storm_year))

# Coordinate selector throws an error when using with Fiona but still generates the graph
storm = storm.sel(lat=[storm_lowerLat,None])

storm_dict = storm.interp().to_dict()

# Some fields are left empty after interpolation, so need to only include ones that will have values
storm_dict_cut = {
    'date':storm_dict['date'],
    'type':storm_dict['type'],
    'lat':storm_dict['lat'],
    'lon':storm_dict['lon'],
}

#storm_df = storm.to_dataframe()
storm_df = pd.DataFrame.from_dict(storm_dict_cut)

storm_df['lon'] = storm_df['lon'].apply(lambda x: x-360.0)

start_date = pytz.utc.localize(storm_df.min()['date'])
end_date = pytz.utc.localize(storm_df.max()['date'])

# Get datasets matching time and location criteria, plus a buffer zone

e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv"
)

e.dataset_id = "allDatasets"

e.variables = [
    "datasetID",
    "minTime",
    "maxTime"
]

e.constraints = {
    "minTime<=": end_date + timedelta(days = dateExtension),
    "maxTime>=": start_date- timedelta(days = dateExtension)
}

datasets_df = e.to_pandas(
    parse_dates=True,
).dropna()
datasets = set(datasets_df["datasetID"])

--> Starting to read in ibtracs data
--> Completed reading in ibtracs data (14.3 seconds)
Using no lon bounds
2019-09-07 12:00:00+00:00
2019-09-09 00:00:00+00:00


In [8]:
# Narrow down datasets that contain the focused variable
# Some datasets may not match at certain points if their coverage is inconsistent


matching_datasets = []
for dataset in datasets:
    info_url = e.get_info_url(dataset_id = dataset, response = "csv")
    dataset_info = pd.read_csv(info_url)

    variable_names = set(dataset_info["Variable Name"].unique())
    if focus_variable in variable_names and dataset not in exclude:
         matching_datasets.append(dataset)

print(matching_datasets)

['SMA_halifax', 'SMA_Holyrood_Buoy2', 'SMA_halifax_fairview', 'SMA_red_island_shoal', 'cna_werc_weather_10-min_avg', 'SMA_st_johns', 'SMA_saint_john', 'SMA_port_aux_basqes_wharf', 'SMA_saint_john_cruise_terminal', 'sma_negl_north_west_river_nlqu0007', 'SMA_halifax_anemometer1', 'SMA_port_aux_basques', 'sma_negl_postville_nlqu0001', 'sma_negl_rigolet_nlqu0002', 'sma_negl_black_tickle_nlqu0003', 'SMA_holyrood_wharf', 'SMA_st_johns_wharf', 'sma_negl_cartwright_junction_nlqu0004', 'SMA_saint_john_wharf', 'DFO_Sutron_KLUMI', 'sma_negl_red_bay_nlqu0005', 'SMA_Fortune_Bay_Buoy', 'SMA_halifax_pier9c', 'SMA_MouthofPlacentiaBayBuoy']


In [ ]:
# Download data from ERDDAP and place it in the station-data folder
# If that dataset already exists, new data will be appended to the existing file

import csv

selected_time=  start_date + timedelta(hours = storm_time_offset)
e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv",
)
        
e.variables = ["time", "longitude", "latitude", focus_variable]

# TODO - Expand the selected time to the storm + buffer? (Or just take entire dataset)

# If data already exists, it won't overwrite
# TODO: Determine if it will update
for dataset in matching_datasets:
    e.dataset_id = dataset
    try:
        buoy_data= e.to_pandas(
                    parse_dates=True,
                ).dropna()
        buoy_data.to_csv(f'../station-data/{dataset}', sep='\t', mode='w')
    except:
        print("couldn't find info for the given dataset for the given time")

In [38]:
# Find most recent values for focus var given a time point
selected_time=  start_date + timedelta(hours = storm_time_offset)
matching_points = pd.DataFrame()

for dataset in matching_datasets:
    try:
        buoy_data= pd.read_csv(f'../station-data/{dataset}', sep='\t')
        buoy_data['time (UTC)'] = pd.to_datetime(buoy_data['time (UTC)'])
        storm_buoy = buoy_data.loc[(buoy_data['time (UTC)'] < selected_time) & (buoy_data['time (UTC)'] >= selected_time - timedelta(hours = storm_time_bounds))]
        # TODO Find out why it's not limiting to time
        recent_row = storm_buoy.tail(1)
        recent_row.insert(0, "dataset", dataset)
        matching_points = pd.concat([matching_points, recent_row.loc[:]]).reset_index(drop=True)
        print("-")
    except:
        print(dataset)
        print("couldn't find info for the given dataset for the given time")
matching_points.set_index('dataset')


2019-09-07 20:00:00+00:00
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-


,Unnamed: 0,time (UTC),longitude (degrees_east),latitude (degrees_north),wind_spd_avg (m s-1),wind_spd_avg (km h-1)
dataset,,,,,,
SMA_halifax,95611,2019-09-07 19:53:01+00:00,-63.544467,44.555900,22.300000,NaN
SMA_Holyrood_Buoy2,11133,2019-09-07 19:53:00+00:00,-53.108107,47.461835,8.600000,NaN
SMA_halifax_fairview,182704,2019-09-07 19:50:00+00:00,-63.627217,44.664167,9.300000,NaN
SMA_red_island_shoal,117453,2019-09-07 19:55:00+00:00,-54.122208,47.318377,10.500000,NaN
cna_werc_weather_10-min_avg,390658,2019-09-07 19:50:00+00:00,-55.682490,46.862990,NaN,0.065
SMA_st_johns,44310,2019-09-07 19:53:01+00:00,-52.630578,47.567160,9.600000,NaN
SMA_saint_john,79757,2019-09-07 19:53:01+00:00,-66.113767,45.197167,14.800000,NaN
SMA_port_aux_basqes_wharf,264126,2019-09-07 19:59:52+00:00,-59.139467,47.574533,11.500000,NaN
SMA_saint_john_cruise_terminal,1237730,2019-09-07 19:59:00+00:00,-66.061167,45.266833,10.800000,NaN


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import cartopy.feature as cfeature
    from cartopy import crs as ccrs
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
except:
    warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")

proj = ccrs.Mercator(central_longitude=305, min_latitude=40, max_latitude=54)
fig, ax = plt.subplots(subplot_kw=dict(projection=proj), figsize=(12,12))
ax.set_extent([312 , 285, 40, 54], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='0.8')
ax.add_feature(cfeature.BORDERS, zorder=10)
ax.add_feature(cfeature.COASTLINE, zorder=10)
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, color='black', alpha=.5, linestyle='--', draw_labels=True)

gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER


map_storm_line = ax.plot(storm_df['lon'],storm_df['lat'], linestyle = 'dashed', linewidth =3, c= 'black', transform=ccrs.PlateCarree())

mask = (storm_df['date'] <= selected_time) & (storm_df['date'] > (selected_time - timedelta(hours=  storm_time_bounds)))
storm_position = storm_df.loc[mask].tail(1)
map_storm_position= ax.scatter(x=storm_position['lon'].iloc[0], y=storm_position['lat'].iloc[0], c= 'red', 
           marker = '*', s=600, transform=ccrs.PlateCarree())

map_stations = ax.scatter(x=matching_points['longitude (degrees_east)'], y=matching_points['latitude (degrees_north)'],  
           c=matching_points[focus_variable + " " + var_units], cmap =  color_map, marker = 'v', s=200, transform=ccrs.PlateCarree(), alpha= 1,
           edgecolors='black')

# TODO: Fix issue where colormap sets to minimum instead of at 0 (or vice versa)

norm = colors.Normalize(matching_points[focus_variable + " " + var_units].min(), 
                        matching_points[focus_variable + " " + var_units].max())
fig.colorbar(cm.ScalarMappable(norm= norm, cmap=color_map), ax=ax)

In [ ]:
# Create plots  for matching datasets
e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv",
)

for dataset in matching_datasets:

    # Still keeps generating graph after dataset fails for some reason?

    e.dataset_id = dataset

    e.constraints = {
        "time>=": start_date - timedelta(days = 2),
        "time<=": end_date + timedelta(days = 2)
    }

    e.variables = ["time", "longitude", "latitude", focus_variable]

    try:
        buoy_data= e.to_pandas(
            parse_dates=True,
        ).dropna()
        buoy_data.plot(x='time (UTC)', y=focus_variable + ' ' + var_units, title=dataset)
    except:
        print("Data does not exist for %s", dataset) 
